# Spatial Joining with fiona and shapely

## Notes 

- Joins longitude and latitude points from a pandas dataframe to shape files polygons (and properties).
- Uses ERSI postcode and lga shape files from the ABS website.
- Includes postcode and local government area options.
- Accuracy can be traded off for speed using the tolerance_setting parameter.


## Importing modules

In [1]:
import fiona
import pandas
from shapely.geometry import Point, asShape
import datetime as dt
from shapely import speedups
speedups.enable()

## Set option for lga or postcode
Note: Shapefiles contain polygon boundaries from 2016, take caution when joining data preceding this time

In [2]:
lga = True
postcode = False

In [3]:
#Default to LGA if both LGA and Postcode are selected.

if lga == False and postcode == False:
    lga = True
    
if lga == True and postcode == True:
    postcode = False

# Conditionally set shape file paths.

if postcode == True:
    shapefile_path = "/users/danielcorcoran/desktop/Partnerships Clean Up/2.1/shapefile data/POA_16/POA_2016_AUST.shp"
    property_code_name = "POA_CODE16"
elif lga == True:
    shapefile_path = "/users/danielcorcoran/desktop/Partnerships Clean Up/2.1/shapefile data/LGA_ABS_16/LGA_2016_AUST.shp"
    property_code_name = "LGA_CODE16" 

## Set input path for file containing coordinates, and export path

In [4]:
#note: the input path is the export path in this case
input_path = "/users/danielcorcoran/Desktop/Partnerships Clean Up/2.1/output files/patent_coordinates_2016.csv"
export_path = "/users/danielcorcoran/desktop/Partnerships Clean Up/2.1/output files/"

## Import data

In [5]:
coord_data = pandas.read_csv(input_path)

## Set column names containing longitude and latitude values 
Check the headers here and set the longitude and latitude column names below

In [6]:
coord_data.head()

,row_index,lat,lon
0,8,-31.953,115.875
1,9,-33.898,151.069
2,10,-27.474,153.004
3,27,-33.681,150.917
4,54,-32.080,115.912


In [7]:
latitude_column_name = "lat"
longitude_column_name = "lon"

## Iterate through fiona collection and store polygons, simplified polygons, properties and boundaries
Creating reference data in the form of lists

In [8]:
#Note: Code is BUILT TO FIT the ABS LGA/Postcode shapefiles released in 2016. 
#Take caution when using data from preceding 2016.

fiona_collection = fiona.open(shapefile_path)
fiona_collection_list = list(fiona_collection)

shapes_list = []
simple_shapes_list = []
shapes_properties_list = []
shapes_bounds_list = []

tolerance_setting = 0.0000005

for index in range(len(fiona_collection_list)):
    
    if lga == True and postcode == False:
        current_shape = fiona_collection_list[index]
        state_name = current_shape["properties"]["STE_NAME16"]
        properties = current_shape["properties"]

        if state_name == "Victoria" and current_shape["geometry"] is not None:
            shapes_list.append(asShape(current_shape["geometry"]))
            simple_shapes_list.append(asShape(current_shape["geometry"]).simplify(tolerance = tolerance_setting))
            shapes_properties_list.append(current_shape["properties"])
            shapes_bounds_list.append(asShape(current_shape["geometry"]).bounds)
            
    elif lga == False and postcode == True:    
        current_shape = fiona_collection_list[index]
        postal_state_code = list(current_shape["properties"]["POA_CODE16"])[0]
        properties = current_shape["properties"]

        if postal_state_code == "3" and current_shape["geometry"] is not None:
            shapes_list.append(asShape(current_shape["geometry"]))
            simple_shapes_list.append(asShape(current_shape["geometry"]).simplify(tolerance = tolerance_setting))
            shapes_properties_list.append(current_shape["properties"])
            shapes_bounds_list.append(asShape(current_shape["geometry"]).bounds)

## Check the size of each list
Checking how many polygons we have in total, there should be 698 postcode polygons, or 80 lga polygons

In [9]:
len(shapes_list)

80

In [10]:
len(shapes_bounds_list)

80

In [11]:
len(simple_shapes_list)

80

In [12]:
len(shapes_properties_list)

80





## Calculate the master bounds
master_bounds corresponds to maximum and minimum coordinates for Victoria. These coordinates are required to speed up algorithm time.

In [13]:
master_bounds = [shapes_bounds_list[0][0], shapes_bounds_list[0][1],
shapes_bounds_list[0][2],shapes_bounds_list[0][3]]

In [14]:
for shape_bounds_tuple in shapes_bounds_list:
    
    current_long_low = shape_bounds_tuple[0]
    current_long_high = shape_bounds_tuple[2]
    current_lat_low = shape_bounds_tuple[1]
    current_lat_high = shape_bounds_tuple[3]
    
    master_long_low = master_bounds[0]
    master_long_high = master_bounds[2]
    master_lat_low = master_bounds[1]
    master_lat_high = master_bounds[3]
    
    if current_long_low < master_long_low:
        master_bounds[0] = current_long_low
        print("Master bounds longitude (low) has been updated to",master_bounds[0])
    
    if current_long_high > master_long_high:
        master_bounds[2] = current_long_high
        print("Master bounds longitude (high) has been updated to",master_bounds[2])
        
    if current_lat_low < master_lat_low:
        master_bounds[1] = current_lat_low
        print("Master bounds latitude (low) has been updated to",master_bounds[1])
        
    if current_lat_high > master_lat_high:
        master_bounds[3] = current_lat_high
        print("Master bounds latitude (high) has been updated to",master_bounds[3])

Master bounds longitude (low) has been updated to 142.353480213
Master bounds latitude (low) has been updated to -37.836583308999934
Master bounds latitude (low) has been updated to -38.67879995399994
Master bounds latitude (high) has been updated to -36.25505162799993
Master bounds latitude (high) has been updated to -35.19282674999994
Master bounds latitude (low) has been updated to -38.85769295599994
Master bounds longitude (high) has been updated to 149.9762909950001
Master bounds longitude (low) has been updated to 140.965735138
Master bounds longitude (low) has been updated to 140.96168199600004
Master bounds latitude (high) has been updated to -33.98042558699996
Master bounds latitude (low) has been updated to -39.13674005499996
Master bounds latitude (low) has been updated to -39.159189527999956


In [15]:
#Check master bounds

master_bounds

[140.96168199600004,
 -39.159189527999956,
 149.9762909950001,
 -33.98042558699996]

## Dictionary to store results

In [16]:
results = {"data_row_index":[],
           "matched_code":[]}

## Process, output results

In [17]:
#Reset counter, contains every coordinate match against a shapefile polygon
contains = 0

#Start timer
start = dt.datetime.now()

#Iterate through coordinate data set, upper level loop
for data_index in range(coord_data.shape[0]):
    
    print("Starting check for geopoint #",data_index)
    
    longitude = coord_data.loc[data_index, longitude_column_name]
    latitude = coord_data.loc[data_index, latitude_column_name]
    point = Point(longitude, latitude)
    
    #Check if point is within the maximum bounds for victoria
    if master_bounds[0] <= longitude <= master_bounds[2] and master_bounds[1] <= latitude <= master_bounds[3]:

        #If within Victorias bounds, start checking against each lga polygon
        for polygon_index in range(len(shapes_bounds_list)):

                #Check if point is within maximum bounds for LGA
                if shapes_bounds_list[polygon_index][0] <= longitude <= shapes_bounds_list[polygon_index][2] and shapes_bounds_list[polygon_index][1] <= latitude <= shapes_bounds_list[polygon_index][3]:

                    #If within LGA box boundary, check if the polygon contains point
                    #simple_shapes_list contains 'reduced' lga polygons
                    if shapes_list[polygon_index].contains(point):

                        print("\t>Coordinates at data index",data_index,"contains point")

                        results["data_row_index"].append(data_index)
                        results["matched_code"].append(shapes_properties_list[polygon_index][property_code_name])
                        contains = contains + 1
                        
                        break

    else:
        print("\t>Shape at data index",data_index,"is outside of Victoria's bounds.")

#Show points found and time taken to run
end = dt.datetime.now()
difference_time = end - start

timestamp = end.strftime("%y%m%d_%H%M")

print("\n","POINTS FOUND:",contains)
print("POINTS TOTAL:",coord_data.shape[0])
print("FINISHED IN:",difference_time)

#Optional : export spine file
#result_dataframe = pandas.DataFrame(results)
#result_dataframe.to_csv("sjoin_output_"+timestamp+".csv", index = False)
#print("Spine data was exported successfully. Use this dataset to join the original data to shape files.")

Starting check for geopoint # 0
	>Shape at data index 0 is outside of Victoria's bounds.
Starting check for geopoint # 1
	>Shape at data index 1 is outside of Victoria's bounds.
Starting check for geopoint # 2
	>Shape at data index 2 is outside of Victoria's bounds.
Starting check for geopoint # 3
	>Shape at data index 3 is outside of Victoria's bounds.
Starting check for geopoint # 4
	>Shape at data index 4 is outside of Victoria's bounds.
Starting check for geopoint # 5
	>Coordinates at data index 5 contains point
Starting check for geopoint # 6
	>Shape at data index 6 is outside of Victoria's bounds.
Starting check for geopoint # 7
	>Shape at data index 7 is outside of Victoria's bounds.
Starting check for geopoint # 8
	>Coordinates at data index 8 contains point
Starting check for geopoint # 9
	>Shape at data index 9 is outside of Victoria's bounds.
Starting check for geopoint # 10
	>Shape at data index 10 is outside of Victoria's bounds.
Starting check for geopoint # 11
	>Coordina

	>Coordinates at data index 116 contains point
Starting check for geopoint # 117
	>Coordinates at data index 117 contains point
Starting check for geopoint # 118
	>Shape at data index 118 is outside of Victoria's bounds.
Starting check for geopoint # 119
	>Shape at data index 119 is outside of Victoria's bounds.
Starting check for geopoint # 120
	>Coordinates at data index 120 contains point
Starting check for geopoint # 121
	>Shape at data index 121 is outside of Victoria's bounds.
Starting check for geopoint # 122
	>Shape at data index 122 is outside of Victoria's bounds.
Starting check for geopoint # 123
	>Shape at data index 123 is outside of Victoria's bounds.
Starting check for geopoint # 124
	>Coordinates at data index 124 contains point
Starting check for geopoint # 125
	>Shape at data index 125 is outside of Victoria's bounds.
Starting check for geopoint # 126
	>Coordinates at data index 126 contains point
Starting check for geopoint # 127
	>Shape at data index 127 is outside 

	>Coordinates at data index 280 contains point
Starting check for geopoint # 281
	>Coordinates at data index 281 contains point
Starting check for geopoint # 282
	>Coordinates at data index 282 contains point
Starting check for geopoint # 283
	>Coordinates at data index 283 contains point
Starting check for geopoint # 284
	>Coordinates at data index 284 contains point
Starting check for geopoint # 285
	>Coordinates at data index 285 contains point
Starting check for geopoint # 286
	>Coordinates at data index 286 contains point
Starting check for geopoint # 287
	>Shape at data index 287 is outside of Victoria's bounds.
Starting check for geopoint # 288
	>Shape at data index 288 is outside of Victoria's bounds.
Starting check for geopoint # 289
	>Shape at data index 289 is outside of Victoria's bounds.
Starting check for geopoint # 290
	>Shape at data index 290 is outside of Victoria's bounds.
Starting check for geopoint # 291
	>Shape at data index 291 is outside of Victoria's bounds.
St

	>Coordinates at data index 397 contains point
Starting check for geopoint # 398
	>Coordinates at data index 398 contains point
Starting check for geopoint # 399
	>Shape at data index 399 is outside of Victoria's bounds.
Starting check for geopoint # 400
Starting check for geopoint # 401
	>Shape at data index 401 is outside of Victoria's bounds.
Starting check for geopoint # 402
	>Shape at data index 402 is outside of Victoria's bounds.
Starting check for geopoint # 403
	>Shape at data index 403 is outside of Victoria's bounds.
Starting check for geopoint # 404
	>Shape at data index 404 is outside of Victoria's bounds.
Starting check for geopoint # 405
	>Coordinates at data index 405 contains point
Starting check for geopoint # 406
	>Shape at data index 406 is outside of Victoria's bounds.
Starting check for geopoint # 407
	>Shape at data index 407 is outside of Victoria's bounds.
Starting check for geopoint # 408
	>Shape at data index 408 is outside of Victoria's bounds.
Starting chec

	>Coordinates at data index 518 contains point
Starting check for geopoint # 519
	>Shape at data index 519 is outside of Victoria's bounds.
Starting check for geopoint # 520
Starting check for geopoint # 521
	>Shape at data index 521 is outside of Victoria's bounds.
Starting check for geopoint # 522
	>Shape at data index 522 is outside of Victoria's bounds.
Starting check for geopoint # 523
	>Shape at data index 523 is outside of Victoria's bounds.
Starting check for geopoint # 524
	>Shape at data index 524 is outside of Victoria's bounds.
Starting check for geopoint # 525
	>Shape at data index 525 is outside of Victoria's bounds.
Starting check for geopoint # 526
	>Shape at data index 526 is outside of Victoria's bounds.
Starting check for geopoint # 527
	>Shape at data index 527 is outside of Victoria's bounds.
Starting check for geopoint # 528
	>Shape at data index 528 is outside of Victoria's bounds.
Starting check for geopoint # 529
	>Shape at data index 529 is outside of Victoria

...

	>Coordinates at data index 7215 contains point
Starting check for geopoint # 7216
	>Coordinates at data index 7216 contains point
Starting check for geopoint # 7217
	>Coordinates at data index 7217 contains point
Starting check for geopoint # 7218
	>Shape at data index 7218 is outside of Victoria's bounds.
Starting check for geopoint # 7219
	>Shape at data index 7219 is outside of Victoria's bounds.
Starting check for geopoint # 7220
	>Coordinates at data index 7220 contains point
Starting check for geopoint # 7221
	>Shape at data index 7221 is outside of Victoria's bounds.
Starting check for geopoint # 7222
	>Shape at data index 7222 is outside of Victoria's bounds.
Starting check for geopoint # 7223
Starting check for geopoint # 7224
	>Shape at data index 7224 is outside of Victoria's bounds.
Starting check for geopoint # 7225
	>Coordinates at data index 7225 contains point
Starting check for geopoint # 7226
	>Shape at data index 7226 is outside of Victoria's bounds.
Starting check 

	>Coordinates at data index 7411 contains point
Starting check for geopoint # 7412
	>Shape at data index 7412 is outside of Victoria's bounds.
Starting check for geopoint # 7413
	>Shape at data index 7413 is outside of Victoria's bounds.
Starting check for geopoint # 7414
	>Shape at data index 7414 is outside of Victoria's bounds.
Starting check for geopoint # 7415
	>Shape at data index 7415 is outside of Victoria's bounds.
Starting check for geopoint # 7416
	>Shape at data index 7416 is outside of Victoria's bounds.
Starting check for geopoint # 7417
	>Shape at data index 7417 is outside of Victoria's bounds.
Starting check for geopoint # 7418
	>Shape at data index 7418 is outside of Victoria's bounds.
Starting check for geopoint # 7419
	>Coordinates at data index 7419 contains point
Starting check for geopoint # 7420
	>Shape at data index 7420 is outside of Victoria's bounds.
Starting check for geopoint # 7421
	>Shape at data index 7421 is outside of Victoria's bounds.
Starting check

	>Coordinates at data index 7502 contains point
Starting check for geopoint # 7503
	>Coordinates at data index 7503 contains point
Starting check for geopoint # 7504
	>Shape at data index 7504 is outside of Victoria's bounds.
Starting check for geopoint # 7505
	>Shape at data index 7505 is outside of Victoria's bounds.
Starting check for geopoint # 7506
	>Coordinates at data index 7506 contains point
Starting check for geopoint # 7507
	>Coordinates at data index 7507 contains point
Starting check for geopoint # 7508
	>Shape at data index 7508 is outside of Victoria's bounds.
Starting check for geopoint # 7509
	>Shape at data index 7509 is outside of Victoria's bounds.
Starting check for geopoint # 7510
	>Shape at data index 7510 is outside of Victoria's bounds.
Starting check for geopoint # 7511
	>Shape at data index 7511 is outside of Victoria's bounds.
Starting check for geopoint # 7512
	>Shape at data index 7512 is outside of Victoria's bounds.
Starting check for geopoint # 7513
	>S

	>Shape at data index 7648 is outside of Victoria's bounds.
Starting check for geopoint # 7649
	>Shape at data index 7649 is outside of Victoria's bounds.
Starting check for geopoint # 7650
	>Shape at data index 7650 is outside of Victoria's bounds.
Starting check for geopoint # 7651
	>Coordinates at data index 7651 contains point
Starting check for geopoint # 7652
	>Shape at data index 7652 is outside of Victoria's bounds.
Starting check for geopoint # 7653
	>Shape at data index 7653 is outside of Victoria's bounds.
Starting check for geopoint # 7654
	>Shape at data index 7654 is outside of Victoria's bounds.
Starting check for geopoint # 7655
	>Shape at data index 7655 is outside of Victoria's bounds.
Starting check for geopoint # 7656
	>Coordinates at data index 7656 contains point
Starting check for geopoint # 7657
	>Shape at data index 7657 is outside of Victoria's bounds.
Starting check for geopoint # 7658
	>Shape at data index 7658 is outside of Victoria's bounds.
Starting check

	>Coordinates at data index 7780 contains point
Starting check for geopoint # 7781
	>Coordinates at data index 7781 contains point
Starting check for geopoint # 7782
	>Shape at data index 7782 is outside of Victoria's bounds.
Starting check for geopoint # 7783
	>Shape at data index 7783 is outside of Victoria's bounds.
Starting check for geopoint # 7784
	>Shape at data index 7784 is outside of Victoria's bounds.
Starting check for geopoint # 7785
	>Shape at data index 7785 is outside of Victoria's bounds.
Starting check for geopoint # 7786
	>Shape at data index 7786 is outside of Victoria's bounds.
Starting check for geopoint # 7787
	>Shape at data index 7787 is outside of Victoria's bounds.
Starting check for geopoint # 7788
	>Shape at data index 7788 is outside of Victoria's bounds.
Starting check for geopoint # 7789
	>Coordinates at data index 7789 contains point
Starting check for geopoint # 7790
	>Shape at data index 7790 is outside of Victoria's bounds.
Starting check for geopoin

## Join coord data to results data using left join

In [18]:
result_data = pandas.DataFrame(results)

In [19]:
result_data.head(5)

,data_row_index,matched_code
0,5,22310
1,8,20740
2,11,24970
3,12,21110
4,26,24970


In [20]:
coord_data_index_list = list(coord_data.index)
coord_data_index_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [21]:
coord_data = pandas.concat([coord_data, pandas.Series(coord_data_index_list).rename("row_index_new")], axis = 1)

In [22]:
coord_data.head(10)

,row_index,lat,lon,row_index_new
0,8,-31.953,115.875,0
1,9,-33.898,151.069,1
2,10,-27.474,153.004,2
3,27,-33.681,150.917,3
4,54,-32.080,115.912,4
5,91,-37.873,145.002,5
6,163,-27.449,153.027,6
7,181,-27.631,151.942,7
8,182,-38.536,145.470,8
9,255,-32.153,115.778,9


In [23]:
final_data = pandas.merge(left = coord_data, 
                          right = result_data,
                            left_on = "row_index_new",
                         right_on = "data_row_index")

In [24]:
final_data.head()

,row_index,lat,lon,row_index_new,data_row_index,matched_code
0,91,-37.873,145.002,5,5,22310
1,182,-38.536,145.470,8,8,20740
2,537,-37.916,145.138,11,11,24970
3,636,-37.822,145.039,12,12,21110
4,1007,-37.902,145.098,26,26,24970


## Export coordinate data with index label

Data should be exported with data_index label to join to output files data_index column

In [25]:
final_data.to_csv(export_path + "patent_coordinates_2016_with_lga.csv", index = False)